<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/algorithms_and_data_structures/sem5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание на 5 лабораторную работу.

Выбрать предметную область. Подобрать и реализовать хеширование (расчет хеш функции). Для разрешения коллизий реализовать метод цепочек на основе списков из 3 лабораторной работы.
Показать работоспособность программы на примере 20 объектов предметной области.

In [28]:
! pip install kagglehub[pandas-datasets]

In [29]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("konstantinalbul/russian-jokes")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/russian-jokes


In [30]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "jokes.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "konstantinalbul/russian-jokes",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())



<ipython-input-30-d133e08cd0cf>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:        theme                                               text  rating
0  pro-sudey  На суде в Стамбуле обвиняемый сказал:\r\n- На...       5
1  pro-sudey  - Вы продолжаете утверждать, что обвиняемый н...       4
2  pro-sudey  На суде.\r\n- Итак, когда дело дошло до столкн...       0
3  pro-sudey  Старую леди сбил автомобиль. На суде ее спраши...       4
4  pro-sudey  Судья говорит:\r\n- Согласно вашей жалобе, об...       2


In [31]:
copy = df.head(20)
print(copy['text'].iloc[3])

Старую леди сбил автомобиль. На суде ее спрашивают, как было
дело. Она:
- Из-за угла вывернул автомобиль. Я влево, и он влево; я
вправо - и он вправо. Тут водитель кричит: "Да стой ты на
месте, черт подери!"
- Я остановилась, тут он на меня и наехал.




In [32]:
# путь функцией хэширования будет данная штука
def sum_hash(text):
    return sum(ord(char) for char in text)

### из предыдущей лабы

In [33]:
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None


class DoublyLinkedList:
    def __init__(self):
        self.head = None  # Начальный (первый) узел
        self.tail = None  # Конечный (последний) узел


    def append(self, data):
        new_node = Node(data)
        if self.head is None:  # Если список пуст
            self.head = self.tail = new_node  # Первый элемент — и голова, и хвост
        else:
            self.tail.next = new_node  # Связываем текущий хвост с новым узлом
            new_node.prev = self.tail  # Связываем новый узел с текущим хвостом
            self.tail = new_node  # Обновляем хвост


    def prepend(self, data):
        new_node = Node(data)
        if self.head is None:  # Если список пуст
            self.head = self.tail = new_node  # Первый элемент — и голова, и хвост
        else:
            new_node.next = self.head  # Новый узел указывает на текущую голову
            self.head.prev = new_node  # Текущая голова знает, что перед ней новый узел
            self.head = new_node  # Обновляем голову


    def delete(self, data):
        if self.head is None:
            return  # Список пуст

        current = self.head
        while current:
            if current.data == data:
                # Если удаляемый элемент - это голова
                if current == self.head:
                    self.head = current.next
                    if self.head:
                        self.head.prev = None  # Обнуляем указатель на предыдущий узел у новой головы
                # Если удаляемый элемент - это хвост
                elif current == self.tail:
                    self.tail = current.prev
                    if self.tail:
                        self.tail.next = None
                else:
                    current.prev.next = current.next
                    current.next.prev = current.prev
                return
            current = current.next


    def traverse(self):
        current = self.head
        while current:
            print(current.data, end=" ")
            current = current.next
        print()

    def reverse_traverse(self):
        current = self.tail
        while current:
            print(current.data, end=" ")
            current = current.prev
        print()

    def find(self, data):
        current = self.head
        while current:
            if current.data == data:
                return current
            current = current.next
        return None  # Элемент не найден




In [ ]:
def sum_hash(text):
    return sum(ord(char) for char in text)

In [36]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [DoublyLinkedList() for _ in range(self.size)]

    def _hash_function(self, key):
        # Используем вашу функцию хеширования
        return sum(ord(char) for char in key) % self.size

    def insert(self, key, value):
        hash_index = self._hash_function(key)
        self.table[hash_index].append((key, value))

    def get(self, key):
        hash_index = self._hash_function(key)
        current_list = self.table[hash_index]
        current_node = current_list.head
        while current_node:
            if current_node.data[0] == key:
                return current_node.data[1]
            current_node = current_node.next
        return None

    def delete(self, key):
        hash_index = self._hash_function(key)
        current_list = self.table[hash_index]
        current_node = current_list.head
        while current_node:
            if current_node.data[0] == key:
                current_list.delete(current_node.data)
                return
            current_node = current_node.next

    def display(self):
        for i in range(self.size):
            print(f"Bucket {i}: ", end="")
            current_list = self.table[i]
            current_node = current_list.head
            while current_node:
                print(current_node.data, end=" -> ")
                current_node = current_node.next
            print("None")

In [38]:
amount_of_jokes = 20
hash_table = HashTable(amount_of_jokes)
for i in range(amount_of_jokes):
    joke_text = copy['text'].iloc[i]
    hash_table.insert(joke_text, f"Joke {i+1}") # Вставляем текст шутки как ключ




In [39]:
hash_table.display()

Bucket 0: ('- Свидетель! - говорит судья. Я прошу вас на минуту забыть,\r\nчто вы работаете в Госкомстате, и говорить суду только\r\nправду.\r\n\r\n', 'Joke 13') -> None
Bucket 1: ('Судья спрашивает четырех индейцев, убежавших из тюрьмы:\r\n- Скажите, почему вы убежали?\r\n- Сидим мы в тюрьме день, второй, третий. На четвертый день\r\nиндеец "Острый глаз" заметил, что одной стены нет.\r\n\r\n', 'Joke 10') -> None
Bucket 2: None
Bucket 3: None
Bucket 4: ('На суде.\r\n- Итак, когда дело дошло до столкновения, у вашего\r\nпротивника в руках была палка. А что было у вас в руках?\r\n- Его жена.\r\n\r\n', 'Joke 3') -> None
Bucket 5: None
Bucket 6: None
Bucket 7: ('Судья:\r\n- Свидетель, вы должны говорить правду, одну только правду и\r\nничего кроме правды.\r\n- Да, господин судья.\r\n- Что в таком случае вы можете сказать по обсуждаемому делу?\r\n- А что можно сказать при таких ограничениях?\r\n\r\n', 'Joke 8') -> None
Bucket 8: ('Судья подсудимому:\r\n- Ну-ну, перестаньте волноваться

In [40]:
# Получаем значение по ключу (пример)
key_to_get = copy['text'].iloc[3]
value = hash_table.get(key_to_get)
print(f"\nValue for key '{key_to_get}': {value}")


Value for key 'Старую леди сбил автомобиль. На суде ее спрашивают, как было
дело. Она:
- Из-за угла вывернул автомобиль. Я влево, и он влево; я
вправо - и он вправо. Тут водитель кричит: "Да стой ты на
месте, черт подери!"
- Я остановилась, тут он на меня и наехал.

': Joke 4


In [45]:
# Удаляем элемент (пример)
key_to_delete = copy['text'].iloc[12]
hash_table.delete(key_to_delete)
print(f"\nAfter deleting key '{key_to_delete}':")


After deleting key '- Свидетель! - говорит судья. Я прошу вас на минуту забыть,
что вы работаете в Госкомстате, и говорить суду только
правду.

':


In [46]:
hash_table.display()

Bucket 0: None
Bucket 1: ('Судья спрашивает четырех индейцев, убежавших из тюрьмы:\r\n- Скажите, почему вы убежали?\r\n- Сидим мы в тюрьме день, второй, третий. На четвертый день\r\nиндеец "Острый глаз" заметил, что одной стены нет.\r\n\r\n', 'Joke 10') -> None
Bucket 2: None
Bucket 3: None
Bucket 4: ('На суде.\r\n- Итак, когда дело дошло до столкновения, у вашего\r\nпротивника в руках была палка. А что было у вас в руках?\r\n- Его жена.\r\n\r\n', 'Joke 3') -> None
Bucket 5: None
Bucket 6: None
Bucket 7: ('Судья:\r\n- Свидетель, вы должны говорить правду, одну только правду и\r\nничего кроме правды.\r\n- Да, господин судья.\r\n- Что в таком случае вы можете сказать по обсуждаемому делу?\r\n- А что можно сказать при таких ограничениях?\r\n\r\n', 'Joke 8') -> None
Bucket 8: ('Судья подсудимому:\r\n- Ну-ну, перестаньте волноваться и расскажите, как это\r\nпроизошло.\r\n- Я ужасно расстроен. А было так. Я сидел и чистил ножичком\r\nапельсин. Тут подошел этот тип, поскользнулся на\r\n